# LCEL chain at work in a typical RAG app

In [75]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [78]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## 让我们看看这个在典型的 RAG 示例中是如何工作的

In [64]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

* 请注意，我们从中心导入的提示有两个变量：“上下文”和“问题”。

In [68]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [63]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a task into smaller, more manageable subtasks in order to facilitate the completion of the overall task.'

#### 让我们详细看看 LCEL 链：
* 正如您所看到的，链的第一部分是 RunnableParallel（请记住，RunnableParallel 可以有多种语法）：

In [65]:
rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [66]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique that breaks down complex tasks into smaller and simpler steps. It involves prompting the model to "think step by step" to make tasks more manageable. This process can be done using techniques like Chain of Thought or Tree of Thoughts.'

* 当我们调用这个链时，它是如何工作的：
    * "什么是任务分解？" 被作为唯一输入传递。
    * `context` 对输入执行检索。
    * format_docs 对输入执行格式化函数。
    * 输入被分配给 `question`。
    * 使用先前的 `question` 和 `context` 变量定义提示。
    * 模型使用先前的提示执行。
    * 输出解析器对模型的响应执行。

#### 注意：之前的格式化函数有什么作用？
`format_docs`函数接受一个名为`docs`的对象列表。这个列表中的每个对象都应该有一个名为`page_content`的属性，该属性存储每个文档的文本内容。

该函数的目的是从`docs`列表中的每个文档中提取`page_content`，然后将这些内容合并为一个单一的字符串。不同文档的内容通过两个换行符（`\n\n`）分隔，这意味着在最终字符串中，每个文档的内容之间会有一个空行。这种格式选择使得合并后的内容更易于阅读，因为它清晰地分隔了不同文档的内容。

以下是该函数工作原理的分解：
1. `for doc in docs`部分遍历`docs`列表中的每个对象。
2. 在每次迭代中，`doc.page_content`访问当前文档的`page_content`属性，该属性包含其文本内容。
3. `join`方法随后将这些文本片段连接成一个单一的字符串，在每个片段之间插入`\n\n`，以确保它们在最终结果中由空行分隔。

该函数最终返回这个新格式化的单一字符串，其中包含所有文档内容，整齐地由空行分隔。